In [14]:
import requests as r
import json
import numpy as np
import pandas as pd
from itertools import product


In [26]:
# Credentials: key, url, headers
url = 'https://api.yelp.com/v3/businesses/search'
key = 'MK4PDFOZPu4hrtlE-hYryaqcGz6WkLQPU9AnK-QccNA-DkVtD3vme9HHMoEXvqH6VnKo_eRcff6V7F9BWj-y-3vJ3XK3KPBD7bWuNXQCmGgSYzsxAM9RuCrTxxcnZXYx'
headers = {
    'Authorization': 'Bearer %s' % key
}

location = ["Manhattan"]
offset = np.arange(0, 1000, 50)
tuples = list(product(location, offset))

info_list = []

def search(cuisine):
  detail_data = []
  detail_info = []
  for loc, step in tuples:
    search_parameters = {
      'term': f'{cuisine} restaurants',
      'location': loc,
      'limit': 50,  # We only need one result to get the total count
      'offset': step
    }
    resp = r.get(url, headers=headers, params=search_parameters)
    raw_data = resp.json()
    detail_info.append(raw_data)

  num_records = detail_info[0]['total']
  print(f"{cuisine} has {num_records} records")

  for info in detail_info:
    data = info['businesses']
    if data:
      for item in data:
        try:
          dict_ = {
                "Id": item['id'],
                "Name": item['name'],
                "Cuisine": cuisine,
                "Address": ' '.join(item['location']['display_address']),
                "Coordinates": item["coordinates"],
                "ReviewCount": item["review_count"],
                "Rating": item['rating'],
                "ZipCode": item["zip_code"]
          }
        except:
            dict_ = {
              "Id": item['id'],
              "Name": item['name'],
              "Cuisine": cuisine,
              "Address": ' '.join(item['location']['display_address']),
              "Coordinates": item["coordinates"],
              "ReviewCount": item["review_count"],
              "Rating": item['rating'],
              "ZipCode": ""
        }
        info_list.append(dict_)

cuisines = ["Chinese", "Japanese", "Italian", "Thai", "Mexican", "French", "American"]
for cuisine in cuisines:
  search(cuisine)

df = pd.DataFrame(info_list)
df["Id"].nunique()

Chinese has 3400 records
Japanese has 1900 records
Italian has 4700 records
Thai has 1800 records
Mexican has 4900 records
French has 803 records
American has 6700 records


5709

In [29]:
df = pd.DataFrame(info_list)
df = df.astype(str)
df["Id"].nunique()
df_unique = df.drop_duplicates(subset='Id', keep='first')

In [28]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [30]:
json_data = df_unique.to_json(orient='records')
with open('restaurants.json', 'w') as f:
    f.write(json_data)
!cp restaurants.json "drive/My Drive/"

In [31]:
json_list = []
df_unique.reset_index(inplace=True)
for index, row in df_unique.iterrows():
    json_list.append(f'{{"index": {{"_index": "restaurants", "_id": {index + 1}}}}}')
    json_list.append(f'{{"Id": "{row["Id"]}", "cuisine": "{row["Cuisine"]}"}}')
output_json = '\n'.join(json_list)
with open('restaurants_opensearch.json', 'w') as f:
    f.write(output_json)
!cp restaurants_opensearch.json "drive/My Drive/"